# So far, the code below which comes from V1 notebook, logs data from FED3 via RbPi board, and sends TTL to the board (to TDT in theory) , in the next step, 

### First I want to make RbPi automatically get data from FEDs as soon as it is switched on (instead of opening the terminal and jupyter lab and running the code)
### Second, I will configure the rbpi pins and the python code to handle 4 FEDs at once

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output (same as your original setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type):
    global pellet_in_well
    if event_type == "Pellet":
        if pellet_in_well:
            GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
            print("Pellet taken, signal turned OFF.")
            pellet_in_well = False  # Update state
            send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
        else:
            print("No pellet was in the well, no signal for pellet taken.")
    elif event_type == "PelletInWell":
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, timestamp):
    print(f"[{timestamp}] Processing event: {event_type}")
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        print("Left poke with pellet, signal triggered.")
        
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        print("Right poke with pellet, signal triggered.")
        
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type)  # Handle PelletInWell and PelletTaken

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            data_list = line.split(",")  # Split the data string into a list
            print(f"Raw FED3 data: {data_list}")  # Debug: Show the full data received
            if len(data_list) == len(column_headers):  # Ensure the data matches the column length
                event_type = data_list[9]  # "Event" field contains event type
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp
                print(f"Extracted event type: {event_type} at {timestamp}")
                process_event(event_type, timestamp)  # Send TTL signal for the event
            else:
                print("Warning: Data length does not match header length, skipping.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


# ChatGPT 01-preview has some suggestions to make sure the code works well when multiple FEDs are included, first I am trying to test the code suggested by ChatGPT


### Key Changes and Improvements:

1. **Thread Safety with Global Variables**:
   - Introduced a `pellet_lock` (`threading.Lock()`) to ensure thread-safe access to the `pellet_in_well` variable.
   - Wrapped access to `pellet_in_well` within `with pellet_lock:` blocks.

2. **Exception Handling for Serial Communication**:
   - Added `try...except` blocks around serial port operations to catch and log exceptions like `serial.SerialException`.
   - Ensured that the serial port is closed properly in the `finally` block.

3. **Graceful Shutdown and GPIO Cleanup**:
   - Used a `stop_event` (`threading.Event()`) to signal threads to stop when a `KeyboardInterrupt` occurs.
   - In the main thread, upon interruption, set the `stop_event` and joined all threads before cleaning up GPIO pins.

4. **Timestamp Accuracy**:
   - Modified the `process_event` function to use the timestamp from the FED3 data (`data_list[0]`) instead of generating a new timestamp. This ensures more accurate event timing.

5. **Data Validation and Parsing**:
   - Added checks to ensure that the data list has at least 10 elements before accessing `data_list[9]` to avoid `IndexError`.
   - Logged warnings when received data is incomplete or does not match expected length.

6. **Resource Management for Serial Ports**:
   - Ensured that serial ports are closed when threads terminate, using `ser.close()` in the `finally` block.

7. **Logging Instead of Print Statements**:
   - Replaced all `print` statements with `logging` calls, providing more control over log levels and formatting.
   - Configured the logging format to include timestamps and log levels.

8. **Error Handling in Threads**:
   - Wrapped the main logic in the `read_from_fed` function within a `try...except` block to catch and log any exceptions that occur within the thread.

9. **Handling Multiple FED Devices**:
   - The code is structured to handle multiple FED devices by starting a new thread for each serial port specified in the `fed_ports` dictionary.
   - Each thread handles its own serial port and shared resources are managed with thread-safe mechanisms.

10. **Graceful Thread Termination**:
    - Threads check the `stop_event` flag in their loops and exit gracefully when the event is set.

11. **Ensuring GPIO Cleanup**:
    - Moved `GPIO.cleanup()` into the `finally` block to ensure it executes regardless of how the script exits.

12. **Code Organization and Modularity**:
    - Organized the code into functions for better readability and maintainability.

### Additional Notes:

- **Serial Port Detection**:
  - The code still uses hardcoded serial ports. For a more robust solution, consider implementing dynamic detection of connected FED devices or using persistent device naming (e.g., by device ID or symbolic links).

- **Exception Handling**:
  - In the `read_from_fed` function, exceptions are logged but the thread continues running unless a critical error occurs, ensuring robustness.

- **Logging Levels**:
  - Adjust the logging level in `logging.basicConfig()` to control the verbosity of the logs (`DEBUG`, `INFO`, `WARNING`, `ERROR`, `CRITICAL`).

- **Python Version**:
  - Added the shebang line `#!/usr/bin/env python3` at the top of the script to specify that it should be run with Python 3.

- **Thread Cleanup**:
  - The main thread waits for all worker threads to finish by calling `t.join()` on each thread after setting the `stop_event`.

- **Resource Management**:
  - The serial port is closed in the `finally` block, ensuring that it is closed even if an error occurs during execution.

Please replace `/dev/ttyACM0` and `FED1` in the `fed_ports` dictionary with the actual serial ports and identifiers for your devices.


In [ ]:

#!/usr/bin/env python3

import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import sys
import traceback

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type):
    global pellet_in_well
    with pellet_lock:
        if event_type == "Pellet":
            if pellet_in_well:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
                logging.debug("Pellet taken, signal turned OFF.")
                pellet_in_well = False  # Update state
                send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
            else:
                logging.debug("No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
            pellet_in_well = True
            logging.debug("Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    logging.info(f"[{timestamp}] Processing event: {event_type}")

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        logging.debug("Left poke event triggered.")

    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        logging.debug("Right poke event triggered.")

    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        logging.debug("Left poke with pellet, signal triggered.")

    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        logging.debug("Right poke with pellet, signal triggered.")

    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type)  # Handle PelletInWell and PelletTaken

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    try:
        try:
            ser = serial.Serial(serial_port, 115200, timeout=1)
            logging.info(f"Opened serial port {serial_port}")
        except serial.SerialException as e:
            logging.error(f"Could not open serial port {serial_port}: {e}")
            return
        except Exception as e:
            logging.error(f"Unexpected error opening serial port {serial_port}: {e}")
            return

        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    logging.debug(f"Raw FED3 data: {data_list}")  # Debug: Show the full data received
                    if len(data_list) >= 10:
                        event_type = data_list[9]  # "Event" field contains event type
                        # Use Raspberry Pi's system time for timestamp
                        logging.debug(f"Extracted event type: {event_type}")
                        process_event(event_type)  # Send TTL signal for the event
                    else:
                        logging.warning("Received incomplete data, skipping this line.")
            except UnicodeDecodeError as e:
                logging.error(f"Decoding error on serial port {serial_port}: {e}")
            except Exception as e:
                logging.error(f"Error reading from serial port {serial_port}: {e}")
                logging.error(traceback.format_exc())
    except Exception as e:
        logging.error(f"Critical error in thread for serial port {serial_port}: {e}")
        logging.error(traceback.format_exc())
    finally:
        try:
            ser.close()
            logging.info(f"Serial port {serial_port} closed.")
        except Exception as e:
            logging.error(f"Error closing serial port {serial_port}: {e}")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

threads = []

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    t = threading.Thread(target=read_from_fed, args=(port,))
    t.start()
    threads.append(t)

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    logging.info("Interrupted by user.")
    stop_event.set()  # Signal threads to stop
finally:
    # Wait for threads to finish
    for t in threads:
        t.join()
    logging.info("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


# The code above suggested by ChatGPT works well (in theory it sends signals to RBPI and I can see the LEDs blinking in response to TTLS)

# In the cell below I want to make the code dynamically detect FED3 device number when we swtich ports/restart the board, etc. This should especially help when we are not connecting the RBPI to a screen and the user has no means of changing the correct port number/name in the code

---

### **Instructions and Notes:**

#### **1. Install Required Python Packages**

Before running the script, ensure that the required Python packages are installed on your Raspberry Pi:

```bash
pip3 install pyudev
```

If you encounter any permission issues, you might need to run the command with `sudo`:

```bash
sudo pip3 install pyudev
```

However, it's generally better to avoid using `sudo` with `pip3` to prevent installing packages as root.

#### **2. Ensure Devices Are Properly Connected**

- **FED3 Devices**: Connect your FED3 devices to the Raspberry Pi via USB. Ensure that they are powered on and that their SD cards are inserted.
- **Auto-Mounting**: The Raspberry Pi should automatically mount the storage partitions of the FED3 devices. You can check this by running:

  ```bash
  lsblk -o NAME,MOUNTPOINT
  ```

  You should see entries corresponding to your FED3 devices with mount points.

#### **3. Permissions**

- **Serial Port Access**: Ensure that the user running the script (e.g., `pi`) is a member of the `dialout` group to access serial ports.

  ```bash
  sudo usermod -a -G dialout pi
  ```

- **Mount Point Access**: Ensure that the script has permission to read the `DeviceNumber.csv` files on the mounted devices. Typically, the `pi` user should have access if the devices are auto-mounted under `/media/pi/`.

#### **4. Test the Script**

- **Run the Script**:

  ```bash
  python3 /path/to/your/script.py
  ```

  Replace `/path/to/your/script.py` with the actual path to the script file.

- **Observe the Logs**: The script will output logs to the console. Look for messages indicating that devices have been mapped and that threads have been started for each device.

- **Interact with FED3 Devices**: Perform some actions on the FED3 devices to generate events. The script should process these events and send the appropriate TTL signals via the GPIO pins.

#### **5. Verify GPIO Output**

- **Hardware Setup**: Ensure that the GPIO pins are connected to your TDT system or another method to monitor the TTL signals.

- **Monitor the Signals**: Use an oscilloscope, logic analyzer, or LED indicators to verify that the TTL signals are being sent as expected.

#### **6. Potential Issues and Troubleshooting**

- **Devices Not Found**: If the script reports "No FED3 devices found," ensure that the devices are connected, powered on, and mounted. Check that the `DeviceNumber.csv` files exist and are accessible.

- **Permissions Errors**: If you encounter permission errors, double-check that the user running the script has the necessary group memberships and access rights.

- **Mount Points Not Detected**: If the script cannot find the mount points of the devices, you might need to adjust the `get_mount_point` function or ensure that the devices are mounted.

- **Attribute Errors**: If the script cannot retrieve attributes like `serial` or `mountpoint`, ensure that the `pyudev` library is working correctly and that the devices expose the necessary information.

#### **7. Adjusting for Your Environment**

- **Mount Points**: If your devices are mounted under a different directory (e.g., `/media/username/`), you may need to adjust the script to reflect that.

- **Serial Numbers and Device Attributes**: If the devices do not expose serial numbers or if the serial numbers do not match between the serial interface and the storage interface, you may need to find alternative attributes to match the devices.

#### **8. Running the Script on Startup**

- Once you have confirmed that the script works as expected, you can set it up to run automatically on boot using a `systemd` service, as previously discussed.

---

### **Final Thoughts**

This script should enable you to automatically detect all connected FED3 devices, map them to their respective device numbers by reading the `DeviceNumber.csv` files, and handle events accordingly, all without the need for manual intervention or connected peripherals.

Please test the script with your devices and let me know if you encounter any issues or need further assistance. I'm here to help!

---

**Note:** Be sure to replace `/path/to/your/script.py` with the actual path where you save the script on your Raspberry Pi.

**Important:** Always ensure that your hardware connections are safe and that you're not risking any damage to your devices or Raspberry Pi when connecting GPIO pins and other peripherals.

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import glob
import pyudev

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per device
pellet_in_well = {}  # Dictionary to keep track per device_number

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, device_number):
    global pellet_in_well
    with pellet_lock:
        if device_number not in pellet_in_well:
            pellet_in_well[device_number] = False
        if event_type == "Pellet":
            if pellet_in_well[device_number]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
                logging.debug(f"Device {device_number}: Pellet taken, signal turned OFF.")
                pellet_in_well[device_number] = False  # Update state
                send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
            else:
                logging.debug(f"Device {device_number}: No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
            pellet_in_well[device_number] = True
            logging.debug(f"Device {device_number}: Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, device_number):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    logging.info(f"[{timestamp}] [Device {device_number}] Processing event: {event_type}")

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        logging.debug(f"Device {device_number}: Left poke event triggered.")

    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        logging.debug(f"Device {device_number}: Right poke event triggered.")

    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        logging.debug(f"Device {device_number}: Left poke with pellet, signal triggered.")

    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        logging.debug(f"Device {device_number}: Right poke with pellet, signal triggered.")

    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, device_number)  # Handle PelletInWell and PelletTaken

# Function to get Device_Number from the mounted device
def get_device_number_from_mount(mount_point):
    device_number_file = os.path.join(mount_point, 'DeviceNumber.csv')
    try:
        with open(device_number_file, 'r') as f:
            device_number = f.read().strip()
            return device_number
    except Exception as e:
        logging.error(f"Error reading DeviceNumber.csv from {mount_point}: {e}")
        return None

# Function to map serial ports to device numbers
def get_device_mappings():
    context = pyudev.Context()
    device_mappings = []

    # Get all serial devices
    serial_devices = [device for device in context.list_devices(subsystem='tty') if 'ttyACM' in device.sys_name or 'ttyUSB' in device.sys_name]

    for serial_device in serial_devices:
        serial_port = serial_device.device_node
        logging.info(f"Found serial device: {serial_port}")

        # Get the parent USB device
        usb_device = serial_device.find_parent('usb', 'usb_device')
        if usb_device is None:
            continue

        # Get the serial number of the USB device
        try:
            usb_serial = usb_device.attributes.asstring('serial').strip()
            logging.info(f"USB serial number: {usb_serial}")
        except AttributeError:
            logging.error(f"Could not get serial number for USB device associated with {serial_port}")
            continue

        # Now, find the mounted storage device with the same serial number
        block_devices = [device for device in context.list_devices(subsystem='block', DEVTYPE='partition')]
        found = False
        for block_device in block_devices:
            try:
                block_usb_device = block_device.find_parent('usb', 'usb_device')
                if block_usb_device:
                    block_usb_serial = block_usb_device.attributes.asstring('serial').strip()
                    if block_usb_serial == usb_serial:
                        mount_point = block_device.attributes.get('mount_point')
                        if not mount_point:
                            # Try to get the mount point from /proc/mounts
                            mount_point = get_mount_point(block_device.device_node)
                        if mount_point:
                            device_number = get_device_number_from_mount(mount_point)
                            if device_number:
                                device_mappings.append({
                                    'serial_port': serial_port,
                                    'device_number': device_number,
                                })
                                logging.info(f"Mapped serial port {serial_port} to device number {device_number}")
                                found = True
                                break
            except Exception as e:
                logging.error(f"Error while processing block device {block_device.device_node}: {e}")
        if not found:
            logging.warning(f"Could not find matching storage device for serial port {serial_port}")
    return device_mappings

# Helper function to get mount point from /proc/mounts
def get_mount_point(device_node):
    try:
        with open('/proc/mounts', 'r') as f:
            mounts = f.readlines()
        for mount in mounts:
            parts = mount.split()
            if parts[0] == device_node:
                return parts[1]
    except Exception as e:
        logging.error(f"Error reading /proc/mounts: {e}")
    return None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, device_number):
    try:
        try:
            ser = serial.Serial(serial_port, 115200, timeout=1)
            logging.info(f"Opened serial port {serial_port} for device {device_number}")
        except serial.SerialException as e:
            logging.error(f"Could not open serial port {serial_port}: {e}")
            return
        except Exception as e:
            logging.error(f"Unexpected error opening serial port {serial_port}: {e}")
            return

        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    logging.debug(f"Device {device_number} raw data: {data_list}")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        logging.debug(f"Device {device_number}: Extracted event type: {event_type}")
                        process_event(event_type, device_number)
                    else:
                        logging.warning(f"Device {device_number}: Received incomplete data, skipping this line.")
                else:
                    # No data received; can implement a sleep or pass
                    pass
            except UnicodeDecodeError as e:
                logging.error(f"Device {device_number}: Decoding error on serial port {serial_port}: {e}")
            except Exception as e:
                logging.error(f"Device {device_number}: Error reading from serial port {serial_port}: {e}")
                logging.error(traceback.format_exc())
        ser.close()
        logging.info(f"Serial port {serial_port} for device {device_number} closed.")
    except Exception as e:
        logging.error(f"Critical error in thread for serial port {serial_port}: {e}")
        logging.error(traceback.format_exc())
    finally:
        try:
            ser.close()
            logging.info(f"Serial port {serial_port} for device {device_number} closed.")
        except Exception as e:
            logging.error(f"Error closing serial port {serial_port}: {e}")

# Main execution
if __name__ == "__main__":
    # Ensure devices are mounted (might need to mount them if not auto-mounted)
    logging.info("Starting device mapping...")
    device_mappings = get_device_mappings()

    if not device_mappings:
        logging.error("No FED3 devices found. Exiting.")
        sys.exit(1)

    threads = []

    # Start threads for each device
    for mapping in device_mappings:
        serial_port = mapping['serial_port']
        device_number = mapping['device_number']
        logging.info(f"Setting up device {device_number} on {serial_port}")
        t = threading.Thread(target=read_from_fed, args=(serial_port, device_number))
        t.start()
        threads.append(t)

    # Cleanup GPIO when script ends
    try:
        while True:
            time.sleep(1)  # Keep the script running
    except KeyboardInterrupt:
        logging.info("Interrupted by user.")
        stop_event.set()  # Signal threads to stop
    finally:
        # Wait for threads to finish
        for t in threads:
            t.join()
        logging.info("Cleaning up GPIO and exiting.")
        GPIO.cleanup()


# Dynamically detecting Device number of FEDs was not good idea, in the cell below I try to only get the USB port number of the RBPI, such that we get data based on Port number and that should help us know from which cage the data is coming

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import pyudev
import re

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}  # Dictionary to keep track per port_identifier

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
                logging.debug(f"{port_identifier}: Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False  # Update state
                send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
            else:
                logging.debug(f"{port_identifier}: No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
            pellet_in_well[port_identifier] = True
            logging.debug(f"{port_identifier}: Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    logging.info(f"[{timestamp}] [{port_identifier}] Processing event: {event_type}")

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        logging.debug(f"{port_identifier}: Left poke event triggered.")

    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        logging.debug(f"{port_identifier}: Right poke event triggered.")

    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        logging.debug(f"{port_identifier}: Left poke with pellet, signal triggered.")

    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        logging.debug(f"{port_identifier}: Right poke with pellet, signal triggered.")

    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier)  # Handle PelletInWell and PelletTaken

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    context = pyudev.Context()
    device_mappings = []

    # List all serial devices (e.g., ttyACM*, ttyUSB*)
    for device in context.list_devices(subsystem='tty'):
        if 'ttyACM' in device.sys_name or 'ttyUSB' in device.sys_name:
            serial_port = device.device_node

            # Find the symlink in /dev/serial/by-path/ that points to this serial port
            symlink = None
            for link in os.listdir('/dev/serial/by-path/'):
                link_path = os.path.join('/dev/serial/by-path/', link)
                if os.path.realpath(link_path) == serial_port:
                    symlink = link
                    break

            if symlink:
                logging.info(f"Serial port {serial_port} has symlink {symlink}")

                # Use the full symlink name as the USB port path
                port_path = symlink

                # Map the symlink to your identifier
                port_identifier = usb_port_mapping.get(port_path)
                if port_identifier:
                    device_mappings.append({
                        'serial_port': serial_port,
                        'port_identifier': port_identifier,
                    })
                    logging.info(f"Mapped serial port {serial_port} to {port_identifier}")
                else:
                    logging.warning(f"No mapping found for USB port {port_path}")
            else:
                logging.warning(f"No symlink found for {serial_port} in /dev/serial/by-path/")
    return device_mappings

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier):
    try:
        try:
            ser = serial.Serial(serial_port, 115200, timeout=1)
            logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        except serial.SerialException as e:
            logging.error(f"Could not open serial port {serial_port}: {e}")
            return
        except Exception as e:
            logging.error(f"Unexpected error opening serial port {serial_port}: {e}")
            return

        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    logging.debug(f"{port_identifier} raw data: {data_list}")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        logging.debug(f"{port_identifier}: Extracted event type: {event_type}")
                        process_event(event_type, port_identifier)
                    else:
                        logging.warning(f"{port_identifier}: Received incomplete data, skipping this line.")
                else:
                    # No data received; can implement a sleep or pass
                    pass
            except UnicodeDecodeError as e:
                logging.error(f"{port_identifier}: Decoding error on serial port {serial_port}: {e}")
            except Exception as e:
                logging.error(f"{port_identifier}: Error reading from serial port {serial_port}: {e}")
                logging.error(traceback.format_exc())
        ser.close()
        logging.info(f"Serial port {serial_port} for {port_identifier} closed.")
    except Exception as e:
        logging.error(f"Critical error in thread for serial port {serial_port}: {e}")
        logging.error(traceback.format_exc())
    finally:
        try:
            ser.close()
            logging.info(f"Serial port {serial_port} for {port_identifier} closed.")
        except Exception as e:
            logging.error(f"Error closing serial port {serial_port}: {e}")

# Main execution
if __name__ == "__main__":
    # Define the USB port to identifier mapping
    usb_port_mapping = {
        # Replace the keys with the actual symlink names from /dev/serial/by-path/
        # For example:
        # 'platform-3f980000.usb-usb-0:1.1:1.0': 'Port 1',
        # 'platform-3f980000.usb-usb-0:1.2:1.0': 'Port 2',
        # Add more mappings as needed
    }

    # Populate usb_port_mapping with actual symlink names
    # You need to fill in the usb_port_mapping dictionary based on your system

    logging.info("Starting device mapping based on USB ports...")
    device_mappings = get_device_mappings_by_usb_port()

    if not device_mappings:
        logging.error("No FED3 devices found. Exiting.")
        sys.exit(1)

    threads = []

    # Start threads for each device
    for mapping in device_mappings:
        serial_port = mapping['serial_port']
        port_identifier = mapping['port_identifier']
        logging.info(f"Setting up {port_identifier} on {serial_port}")
        t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier))
        t.start()
        threads.append(t)

    # Cleanup GPIO when script ends
    try:
        while True:
            time.sleep(1)  # Keep the script running
    except KeyboardInterrupt:
        logging.info("Interrupted by user.")
        stop_event.set()  # Signal threads to stop
    finally:
        # Wait for threads to finish
        for t in threads:
            t.join()
        logging.info("Cleaning up GPIO and exiting.")
        GPIO.cleanup()
